# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [21]:
# 1) 
    # Pegar dados em JSON
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
dados = requests.get(url)
dados = dados.json()

    # Criar e filtrar dados para o estdo do Rio de Janeiro.
df = pd.DataFrame(dados['items']) # Acessar dados no json.
df = df.loc[df['uf'] == 'RJ'].reset_index(drop=True) # Filtro.
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,42252,Município,Itatiaia,3302254,RJ,PVL02.000042/2020-82,Encaminhado à PGFN com manifestação técnica fa...,17944.101489/2020-31,2020-05-19T13:29:59Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,30000000.0,1,1,21/05/2020
1,56075,Município,Resende,3304201,RJ,PVL02.003504/2022-85,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,2160000.0,0,1,28/07/2022
2,38893,Município,Paty do Alferes,3303856,RJ,PVL02.007600/2019-05,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,3512000.0,0,1,30/10/2019
3,33609,Município,Vassouras,3306206,RJ,PVL02.002481/2019-96,Deferido,17944.102288/2019-17,2019-07-29T17:06:25Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,40500000.0,0,1,31/07/2019
4,65267,Município,Miguel Pereira,3302908,RJ,PVL02.002355/2023-18,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,10000000.0,0,1,13/07/2023


In [23]:
# 2) # Selecionando os 3 primeiros elementos com maior frequência.
df['status'].value_counts()[:3]

status
Deferido                                                 35
Encaminhado à PGFN com manifestação técnica favorável     9
Deferido (PVL-IF)                                         8
Name: count, dtype: int64

In [34]:
# 3) Passando dados para o formato datetime e selecionando o ano. 
df['Ano_status'] = pd.to_datetime(df['data_status'], format='%d/%m/%Y').dt.year 

In [36]:
df['Ano_status'].value_counts() # Frequência.

Ano_status
2008    10
2012     9
2010     6
2019     6
2023     6
2011     6
2020     5
2007     5
2013     5
2014     3
2017     3
2015     3
2022     2
2002     2
2009     2
2006     1
2016     1
2018     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [56]:
#1) 

def consulta(estado: str, interessado: str) -> list:
    """
    Função desenvolvida para extrair os dados da URL, transforma-los em dataframe e então filtrar os dados com base na entrada.
    estado => string: Qual o estado da federação;
    interessado => string: Cidade ou Estado que está interessado.
    
    """
        # Pegar dados em JSON
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    dados = requests.get(url)
    dados = dados.json()

        # Criar e filtrar dados para o estdo do Rio de Janeiro.
    df_saida = pd.DataFrame(dados['items']) # Acessar dados no json.
        
        # Retorna os dados filtrados.
    return df_saida.loc[(df_saida['uf'] == estado) & (df_saida['tipo_interessado'] == interessado)]

df_SP = consulta('SP','Estado') # Chamando a funçaõ
df_SP.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
680,65870,Estado,São Paulo,35,SP,PVL02.002226/2023-20,Encaminhado à PGFN com manifestação técnica fa...,17944.102389/2023-74,2023-07-24T14:27:37Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.801335e+08,0,0,17/08/2023
733,64331,Estado,São Paulo,35,SP,PVL02.000813/2023-84,Encaminhado à PGFN com manifestação técnica fa...,17944.101906/2023-98,2023-05-25T19:18:15Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.500000e+09,0,0,30/05/2023
936,11126,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001366/2009-95,2009-11-19T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.809580e+08,1,0,25/11/2009
1643,4931,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,10951.001032/2003-98,2003-12-12T02:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,2.000000e+07,1,0,07/07/2004
1655,5223,Estado,São Paulo,35,SP,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000063/2010-99,2010-08-18T03:00:00Z,Operação contratual externa (com garantia da U...,Preservação e recuperação ambiental,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,1.624540e+08,1,0,25/08/2010


In [71]:
# 2) 
df_MG = consulta('MG','Estado') # Chamando a função
consultas_mg_estado = df_MG.loc[df_MG['status'] == 'Arquivado por decurso de prazo'].shape[0] # Filtro
print(f"\n{'-'*5} Para o estado de Minas Gerais, sendo interessado pelo estado, foram registrados {consultas_mg_estado} {'-'*5}\n")


----- Para o estado de Minas Gerais, sendo interessado pelo estado, foram registrados 1 -----



In [70]:
# 3) 
df_BA = consulta('BA','Município')
cidade = df_BA['interessado'].value_counts()[:1]
print(f"\nA cidade de {cidade.index[0]} é a que tem mais pedidos, contendo um total de {cidade[0]}\n")


A cidade de Amargosa é a que tem mais pedidos, contendo um total de 6



In [69]:
# 4) 
df_BA = consulta('BA','Estado') 
df_BA.to_csv('C:\\Users\\alcid\\GitHub\\Portfolio\\EBAC\\Dados\\dados_bahia.csv')